In [34]:
# Dashboard code
import os
import warnings
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz 
import hvplot.pandas
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

load_dotenv()

True

In [35]:
# Set Alpaca API key and secret
# YOUR CODE HERE!
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
# Create the Alpaca API object
# YOUR CODE HERE!
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [43]:
# Define the timezone: America/New_York
tz = pytz.timezone('America/New_York')

# Get the current datetime in the specified timezone
now = datetime.now(tz)

correct_date = now - timedelta(days=1)

# Convert to pandas Timestamp with timezone and format as ISO 8601 string
# Create a Timestamp object from the correct_date
correct_timestamp = pd.Timestamp(correct_date)

# Normalize the timestamp to midnight (00:00:00)
correct_timestamp_normalized = correct_timestamp.normalize()

# Convert the localized Timestamp object to the desired format
correct_date_str = correct_timestamp_normalized.isoformat()

# Set the tickers
tickers = ["SPY", "QQQ", "IWM", "AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "JNJ", "JPM"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# YOUR CODE HERE!
current_portfolio_df = api.get_bars(tickers, timeframe, start=correct_date_str, end=correct_date_str).df



                            close      high      low  trade_count    open  \
timestamp                                                                   
2024-05-07 04:00:00+00:00  182.40  184.9000  181.320       747523  183.45   
2024-05-07 04:00:00+00:00  188.76  189.9400  187.305       400760  188.92   
2024-05-07 04:00:00+00:00  171.25  171.7600  168.390       356117  168.50   
2024-05-07 04:00:00+00:00  204.97  206.1500  204.500       144799  204.77   
2024-05-07 04:00:00+00:00  148.72  149.7300  148.450        95891  149.38   
2024-05-07 04:00:00+00:00  191.75  192.9300  191.650       115145  191.70   
2024-05-07 04:00:00+00:00  468.24  471.5300  461.310       252772  466.29   
2024-05-07 04:00:00+00:00  409.34  414.6700  409.090       334648  414.66   
2024-05-07 04:00:00+00:00  905.54  917.8099  823.250       835462  910.98   
2024-05-07 04:00:00+00:00  440.32  441.9700  439.580       286484  440.70   
2024-05-07 04:00:00+00:00  517.14  518.5700  516.450       432308  517.56   

In [44]:
if 'symbol' in current_portfolio_df.columns:
    # Separate data by ticker and drop the 'symbol' column
    spy_df = current_portfolio_df[current_portfolio_df['symbol'] == 'SPY'].drop(columns='symbol', errors='ignore')
    qqq_df = current_portfolio_df[current_portfolio_df['symbol'] == 'QQQ'].drop(columns='symbol', errors='ignore')
    iwm_df = current_portfolio_df[current_portfolio_df['symbol'] == 'IWM'].drop(columns='symbol', errors='ignore')
    aapl_df = current_portfolio_df[current_portfolio_df['symbol'] == 'AAPL'].drop(columns='symbol', errors='ignore')
    msft_df = current_portfolio_df[current_portfolio_df['symbol'] == 'MSFT'].drop(columns='symbol', errors='ignore')
    amzn_df = current_portfolio_df[current_portfolio_df['symbol'] == 'AMZN'].drop(columns='symbol', errors='ignore')
    googl_df = current_portfolio_df[current_portfolio_df['symbol'] == 'GOOGL'].drop(columns='symbol', errors='ignore')
    meta_df = current_portfolio_df[current_portfolio_df['symbol'] == 'META'].drop(columns='symbol', errors='ignore')
    tsla_df = current_portfolio_df[current_portfolio_df['symbol'] == 'TSLA'].drop(columns='symbol', errors='ignore')
    nvda_df = current_portfolio_df[current_portfolio_df['symbol'] == 'NVDA'].drop(columns='symbol', errors='ignore')
    jnj_df = current_portfolio_df[current_portfolio_df['symbol'] == 'JNJ'].drop(columns='symbol', errors='ignore')
    jpm_df = current_portfolio_df[current_portfolio_df['symbol'] == 'JPM'].drop(columns='symbol', errors='ignore')

    # Concatenate the ticker DataFrames along the columns axis
    portfolio_concat_df = pd.concat([spy_df, qqq_df, iwm_df, aapl_df, msft_df, amzn_df, googl_df, meta_df, tsla_df, nvda_df, jnj_df, jpm_df],
                                    axis=1, 
                                    keys=['SPY', 'QQQ', 'IWM', 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JNJ', 'JPM'])
    print(portfolio_concat_df.head())
else:
    print("Symbol column not found in DataFrame.")


                              SPY                                      \
                            close    high     low trade_count    open   
timestamp                                                               
2024-05-07 04:00:00+00:00  517.14  518.57  516.45      432308  517.56   

                                                    QQQ                  ...  \
                             volume        vwap   close    high     low  ...   
timestamp                                                                ...   
2024-05-07 04:00:00+00:00  53563924  517.483354  440.32  441.97  439.58  ...   

                              JNJ                         JPM                  \
                             open   volume       vwap   close    high     low   
timestamp                                                                       
2024-05-07 04:00:00+00:00  149.38  7333732  148.84203  191.75  192.93  191.65   

                                                             

In [47]:
# Extract the closing price, volume, and VWAP for each stock
close_price = portfolio_concat_df.loc[:, (slice(None), 'close')]
volume = portfolio_concat_df.loc[:, (slice(None), 'volume')]
vwap = portfolio_concat_df.loc[:, (slice(None), 'vwap')]

# Print the summary statistics for each stock
print("Summary Statistics:")
for stock in close_price.columns.get_level_values(0).unique():
    print(f"{stock}:")
    print(f"  Closing Price: {close_price.loc[:, (stock, 'close')].values[0]:.2f}")
    print(f"  Volume: {volume.loc[:, (stock, 'volume')].values[0]:,.0f}")
    print(f"  VWAP: {vwap.loc[:, (stock, 'vwap')].values[0]:.2f}")


Summary Statistics:
SPY:
  Closing Price: 517.14
  Volume: 53,563,924
  VWAP: 517.48
QQQ:
  Closing Price: 440.32
  Volume: 31,738,096
  VWAP: 440.83
IWM:
  Closing Price: 204.97
  Volume: 20,116,904
  VWAP: 205.32
AAPL:
  Closing Price: 182.40
  Volume: 77,399,199
  VWAP: 182.78
MSFT:
  Closing Price: 409.34
  Volume: 20,165,102
  VWAP: 410.79
AMZN:
  Closing Price: 188.76
  Volume: 34,049,404
  VWAP: 188.75
GOOGL:
  Closing Price: 171.25
  Volume: 28,133,194
  VWAP: 170.71
META:
  Closing Price: 468.24
  Volume: 13,406,818
  VWAP: 467.13
TSLA:
  Closing Price: 177.81
  Volume: 75,046,028
  VWAP: 179.21
NVDA:
  Closing Price: 905.54
  Volume: 43,752,054
  VWAP: 905.68
JNJ:
  Closing Price: 148.72
  Volume: 7,333,732
  VWAP: 148.84
JPM:
  Closing Price: 191.75
  Volume: 7,827,773
  VWAP: 192.15


In [55]:
# Compare closing price with VWAP for each stock and calculate distance from VWAP as a percentage
above_vwap = {}
below_vwap = {}
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    close_price = portfolio_concat_df.loc[:, (stock, 'close')].values[0]
    vwap_price = portfolio_concat_df.loc[:, (stock, 'vwap')].values[0]
    distance_from_vwap_pct = (close_price - vwap_price) / vwap_price * 100
    if close_price > vwap_price:
        above_vwap[stock] = distance_from_vwap_pct
    else:
        below_vwap[stock] = distance_from_vwap_pct

# Print the stocks trading above their VWAP and the distance as a percentage
print("Stocks trading above VWAP:")
for stock, distance_pct in above_vwap.items():
    print(f"{stock}: Above VWAP by {distance_pct:.2f}%")

# Print the stocks trading below their VWAP and the distance as a percentage
print("\nStocks trading below VWAP:")
for stock, distance_pct in below_vwap.items():
    print(f"{stock}: Below VWAP by {distance_pct:.2f}%")

# Insights
print("\nInsights:")
if len(above_vwap) > len(below_vwap):
    print("- The majority of stocks are trading above their VWAP, indicating a bullish sentiment in the market.")
else:
    print("- The majority of stocks are trading below their VWAP, indicating a bearish sentiment in the market.")

max_above_stock = max(above_vwap, key=above_vwap.get)
max_above_distance_pct = above_vwap[max_above_stock]
print(f"- {max_above_stock} has the highest positive distance from VWAP, trading {max_above_distance_pct:.2f}% above its VWAP.")

max_below_stock = min(below_vwap, key=below_vwap.get)
max_below_distance_pct = below_vwap[max_below_stock]
print(f"- {max_below_stock} has the highest negative distance from VWAP, trading {max_below_distance_pct:.2f}% below its VWAP.")


Stocks trading above VWAP:
AMZN: Above VWAP by 0.01%
GOOGL: Above VWAP by 0.32%
META: Above VWAP by 0.24%

Stocks trading below VWAP:
SPY: Below VWAP by -0.07%
QQQ: Below VWAP by -0.12%
IWM: Below VWAP by -0.17%
AAPL: Below VWAP by -0.21%
MSFT: Below VWAP by -0.35%
TSLA: Below VWAP by -0.78%
NVDA: Below VWAP by -0.02%
JNJ: Below VWAP by -0.08%
JPM: Below VWAP by -0.21%

Insights:
- The majority of stocks are trading below their VWAP, indicating a bearish sentiment in the market.
- GOOGL has the highest positive distance from VWAP, trading 0.32% above its VWAP.
- TSLA has the highest negative distance from VWAP, trading -0.78% below its VWAP.


In [60]:

# Calculate the high-low range for each stock
portfolio_concat_df['SPY', 'high_low_range'] = portfolio_concat_df['SPY', 'high'] - portfolio_concat_df['SPY', 'low']
portfolio_concat_df['QQQ', 'high_low_range'] = portfolio_concat_df['QQQ', 'high'] - portfolio_concat_df['QQQ', 'low']
portfolio_concat_df['JNJ', 'high_low_range'] = portfolio_concat_df['JNJ', 'high'] - portfolio_concat_df['JNJ', 'low']
portfolio_concat_df['JPM', 'high_low_range'] = portfolio_concat_df['JPM', 'high'] - portfolio_concat_df['JPM', 'low']

# Calculate the high-low range as a percentage of the opening price for each stock
portfolio_concat_df['SPY', 'high_low_range_pct'] = portfolio_concat_df['SPY', 'high_low_range'] / portfolio_concat_df['SPY', 'open'] * 100
portfolio_concat_df['QQQ', 'high_low_range_pct'] = portfolio_concat_df['QQQ', 'high_low_range'] / portfolio_concat_df['QQQ', 'open'] * 100
portfolio_concat_df['JNJ', 'high_low_range_pct'] = portfolio_concat_df['JNJ', 'high_low_range'] / portfolio_concat_df['JNJ', 'open'] * 100
portfolio_concat_df['JPM', 'high_low_range_pct'] = portfolio_concat_df['JPM', 'high_low_range'] / portfolio_concat_df['JPM', 'open'] * 100

# Print the high-low range and percentage
print("High-Low Price Range and Percentage:")
range_df = portfolio_concat_df.loc[:, (slice(None), ['high_low_range', 'high_low_range_pct'])]
range_df.columns = range_df.columns.droplevel(1)
print(range_df)

# Identify the stock with the highest and lowest high-low range percentage
max_range_pct_stock = range_df.iloc[0].idxmax()
min_range_pct_stock = range_df.iloc[0].idxmin()
print(f"\nStock with the highest high-low range percentage: {max_range_pct_stock}")
print(f"Stock with the lowest high-low range percentage: {min_range_pct_stock}")

# Calculate the average high-low range percentage across all stocks
avg_range_pct = range_df.iloc[0].mean()
print(f"\nAverage high-low range percentage across all stocks: {avg_range_pct:.2f}%")

# Identify stocks with above-average and below-average high-low range percentages
above_avg_stocks = range_df.columns[range_df.iloc[0] > avg_range_pct].tolist()
below_avg_stocks = range_df.columns[range_df.iloc[0] < avg_range_pct].tolist()
print("\nStocks with above-average high-low range percentage:", ', '.join(above_avg_stocks))
print("Stocks with below-average high-low range percentage:", ', '.join(below_avg_stocks))


High-Low Price Range and Percentage:
                            SPY   QQQ   JNJ   JPM       SPY       QQQ  \
timestamp                                                               
2024-05-07 04:00:00+00:00  2.12  2.39  1.28  1.28  0.409614  0.542319   

                                JNJ      JPM  
timestamp                                     
2024-05-07 04:00:00+00:00  0.856875  0.66771  

Stock with the highest high-low range percentage: QQQ
Stock with the lowest high-low range percentage: SPY

Average high-low range percentage across all stocks: 1.19%

Stocks with above-average high-low range percentage: SPY, QQQ, JNJ, JPM
Stocks with below-average high-low range percentage: SPY, QQQ, JNJ, JPM


In [56]:
# Calculate total volume for each stock
total_volume = portfolio_concat_df.loc[:, (slice(None), 'volume')].sum()

# Sort the stocks by total volume in descending order
volume_ranking = total_volume.sort_values(ascending=False)

# Format volume numbers with commas
volume_ranking_formatted = volume_ranking.apply(lambda x: f"{x:,.0f}")

# Print the volume ranking
print("Volume Ranking:")
print(volume_ranking_formatted)

# Identify the top 3 most actively traded stocks
top_traded_stocks = volume_ranking.index.get_level_values(0)[:3].tolist()
print(f"\nTop 3 Most Actively Traded Stocks: {', '.join(top_traded_stocks)}")


Volume Ranking:
AAPL   volume    77,399,199
TSLA   volume    75,046,028
SPY    volume    53,563,924
NVDA   volume    43,752,054
AMZN   volume    34,049,404
QQQ    volume    31,738,096
GOOGL  volume    28,133,194
MSFT   volume    20,165,102
IWM    volume    20,116,904
META   volume    13,406,818
JPM    volume     7,827,773
JNJ    volume     7,333,732
dtype: object

Top 3 Most Actively Traded Stocks: AAPL, TSLA, SPY


In [41]:
# Calculate the average trade size for each security
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    portfolio_concat_df.loc[:, (stock, 'avg_trade_size')] = portfolio_concat_df.loc[:, (stock, 'volume')] / portfolio_concat_df.loc[:, (stock, 'trade_count')]

# Identify securities with lower trade count compared to volume
institutional_activity = {}
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    if portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0] > 100:
        institutional_activity[stock] = portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0]

print("Securities with potential institutional activity:")
for stock, avg_trade_size in institutional_activity.items():
    print(f"{stock}: Average trade size = {avg_trade_size:.2f} shares")

# Identify securities with volume closer to trade count
retail_activity = {}
for stock in portfolio_concat_df.columns.get_level_values(0).unique():
    if portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0] <= 100:
        retail_activity[stock] = portfolio_concat_df.loc[:, (stock, 'avg_trade_size')].values[0]

print("\nSecurities with potential retail activity:")
for stock, avg_trade_size in retail_activity.items():
    print(f"{stock}: Average trade size = {avg_trade_size:.2f} shares")


Securities with potential institutional activity:
SPY: Average trade size = 123.90 shares
QQQ: Average trade size = 110.78 shares
IWM: Average trade size = 138.93 shares
AAPL: Average trade size = 103.54 shares

Securities with potential retail activity:
MSFT: Average trade size = 60.26 shares
AMZN: Average trade size = 84.96 shares
GOOGL: Average trade size = 79.00 shares
META: Average trade size = 53.04 shares
TSLA: Average trade size = 79.64 shares
NVDA: Average trade size = 52.37 shares
JNJ: Average trade size = 76.48 shares
JPM: Average trade size = 67.98 shares


In [42]:
# Calculate the start date (3 months ago)
start_date = now - timedelta(days=90)

# Convert to pandas Timestamp with timezone and format as ISO 8601 string
start_timestamp = pd.Timestamp(start_date).normalize().isoformat()
end_timestamp = pd.Timestamp(now).normalize().isoformat()

# Set the tickers
tickers = ["QQQ", "SPY", "IWM", "XLP", "XLV", "XLF", "XLE", "XLK", "XLY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get historical closing prices for the tickers
historical_data_df = api.get_bars(tickers, timeframe, start=start_timestamp, end=end_timestamp).df

# Reorganize the DataFrame
historical_data_df = historical_data_df.reset_index()
historical_data_df['timestamp'] = pd.to_datetime(historical_data_df['timestamp'])

# Pivot the DataFrame to have tickers as columns
pivoted_df = historical_data_df.pivot(index='timestamp', columns='symbol', values='close')

# Normalize the prices by dividing by the first price for each ticker
normalized_df = pivoted_df.div(pivoted_df.iloc[0])

# Create a line plot using hvplot
line_plot = normalized_df.hvplot.line(xlabel='Date', ylabel='Relative Price', width=800, height=400, legend='top')

# Display the plot
line_plot

:NdOverlay   [symbol]
   :Curve   [timestamp]   (value)

In [33]:
# Define the directory path for raw_data
raw_data_dir = '../data/raw_data'
if not os.path.exists(raw_data_dir):
    os.makedirs(raw_data_dir)

# Function to save DataFrame to CSV or log if empty
def save_data(df, filename, description):
    if not df.empty:
        df.to_csv(filename)
        print(f"{description} data saved successfully to {filename}")
    else:
        print(f"No {description} data to save. Writing null placeholder.")
        # Write a null placeholder to indicate no data was fetched
        with open(filename, 'w') as file:
            file.write('null')

try:
    # Check if the DataFrame exists and isn't empty
    if 'portfolio_concat_df' in locals() and not portfolio_concat_df.empty:
        stock_file_path = os.path.join(raw_data_dir, 'stock_data.csv')
        save_data(portfolio_concat_df, stock_file_path, 'Stock')
    else:
        save_data(pd.DataFrame(), os.path.join(raw_data_dir, 'stock_data.csv'), 'Stock')

    if 'crypto_concat_df' in locals() and not crypto_concat_df.empty:
        crypto_file_path = os.path.join(raw_data_dir, 'crypto_data.csv')
        save_data(crypto_concat_df, crypto_file_path, 'Crypto')
    else:
        save_data(pd.DataFrame(), os.path.join(raw_data_dir, 'crypto_data.csv'), 'Crypto')
except Exception as e:
    print(f"An error occurred while saving data: {str(e)}")


Stock data saved successfully to ../data/raw_data\stock_data.csv
No Crypto data to save. Writing null placeholder.
